In [3]:
!pip install pyswarm

  Preparing metadata (setup.py) ... done
  Created wheel for pyswarm: filename=pyswarm-0.6-py3-none-any.whl size=4464 sha256=a5ff07b2cca91fc0e7b664e134c7ebaf9bc92ac5f4fe579a3dcffb48868bcb24
  Stored in directory: /root/.cache/pip/wheels/71/67/40/62fa158f497f942277cbab8199b05cb61c571ab324e67ad0d6
Successfully built pyswarm


In [4]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.metrics import accuracy_score, mean_squared_error
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from pyswarm import pso

In [5]:
from google.colab import files
upload = files.upload()

Saving seattle_weather.csv to seattle_weather.csv


In [7]:
# Access the uploaded file directly
data = pd.read_csv("seattle_weather.csv")

In [9]:
# Convert date to datetime with correct format
data['date'] = pd.to_datetime(data['date'], format='%d/%m/%Y')
# Drop any rows with missing values
data.dropna(inplace=True)
# Encode categorical variable 'weather' using LabelEncoder
label_encoder = LabelEncoder()
data['weather'] = label_encoder.fit_transform(data['weather'])
# Split the dataset into features and target
X = data.drop(['date', 'weather'], axis=1)
y = data['weather']
# Split the data into train and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
# Standardize the features
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)


In [10]:
def create_nn_model(input_shape):
    model = Sequential([
        Dense(64, activation='relu', input_shape=(input_shape,)),
        Dense(32, activation='relu'),
        Dense(16, activation='relu'),
        Dense(4, activation='softmax')
    ])
    model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])
    return model


In [12]:
# Encode categorical variable 'weather' using LabelEncoder
label_encoder = LabelEncoder()
y = label_encoder.fit_transform(data['weather'])

In [14]:
print(data['weather'].unique())

[0 2 4 3 1]


In [15]:
print("Training Labels - Min:", np.min(y_train), "Max:", np.max(y_train))
print("Testing Labels - Min:", np.min(y_test), "Max:", np.max(y_test))

Training Labels - Min: 0 Max: 4
Testing Labels - Min: 0 Max: 4


In [16]:
import numpy as np
from pyswarm import pso


In [17]:
def pso_objective_function(hyperparameters):
    # Extract hyperparameters
    learning_rate, num_layers, num_neurons = hyperparameters

    # Create and compile the neural network model using hyperparameters
    model = create_nn_model(X_train_scaled.shape[1], learning_rate, num_layers, num_neurons)
    model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

    # Train the model
    model.fit(X_train_scaled, y_train, epochs=10, batch_size=32, verbose=0)

    # Evaluate the model on validation data
    loss, accuracy = model.evaluate(X_validation_scaled, y_validation)

    # Return negative accuracy as PSO minimizes the objective function
    return -accuracy


In [18]:
# Bounds for hyperparameters: (learning_rate, num_layers, num_neurons)
lb = [0.001, 1, 16]  # Lower bounds
ub = [0.1, 5, 256]    # Upper bounds


In [21]:
def create_nn_model(input_shape, learning_rate, num_layers, num_neurons):
    model = Sequential()
    model.add(Dense(num_neurons, input_shape=(input_shape,), activation='relu'))
    for _ in range(num_layers - 1):
        model.add(Dense(num_neurons, activation='relu'))
    model.add(Dense(5, activation='softmax'))  # Assuming 5 output classes

    optimizer = tf.keras.optimizers.Adam(learning_rate=learning_rate)
    model.compile(optimizer=optimizer, loss='sparse_categorical_crossentropy', metrics=['accuracy'])

    return model


In [23]:
def pso_objective_function(hyperparameters):
    learning_rate, num_layers, num_neurons = hyperparameters

    # Create and compile the neural network model using hyperparameters
    model = create_nn_model(X_train_scaled.shape[1], learning_rate, num_layers, num_neurons)

    # Train the model
    model.fit(X_train_scaled, y_train, epochs=10, batch_size=32, verbose=0)

    # Evaluate the model on validation data
    loss, accuracy = model.evaluate(X_validation_scaled, y_validation)

    # Return negative accuracy as PSO minimizes the objective function
    return -accuracy


In [26]:
import numpy as np

print(np.unique(y_test))

[0 1 2 3 4]


In [37]:
from keras.models import Sequential
from keras.layers import Dense

# Define your baseline model
baseline_model = Sequential([
    Dense(128, activation='relu', input_shape=(X_train_scaled.shape[1],)),
    Dense(64, activation='relu'),
    Dense(num_classes, activation='softmax')
])

# Compile the model
baseline_model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

# Evaluate the baseline model
baseline_loss, baseline_accuracy = baseline_model.evaluate(X_test_scaled, y_test)
print("Baseline Model Accuracy:", baseline_accuracy)


10/10 [==============================] - 2s 9ms/step - loss: 1.6785 - accuracy: 0.3174
Baseline Model Accuracy: 0.31740614771842957


In [39]:
def optimize_nn_model(params):
    """
    Function to optimize neural network model using PSO.

    Args:
    - params (list): List containing the parameters to be optimized.

    Returns:
    - loss (float): Loss value of the model.
    """
    # Extract parameters
    hidden_layer_sizes = [int(param) for param in params]

    # Initialize and train the MLPClassifier
    mlp_clf = MLPClassifier(hidden_layer_sizes=hidden_layer_sizes, activation='relu', max_iter=1000, random_state=42)
    mlp_clf.fit(X_train_scaled, y_train)

    # Evaluate the model
    y_pred = mlp_clf.predict(X_test_scaled)
    loss = 1 - accuracy_score(y_test, y_pred)  # 1 - accuracy gives us loss

    return loss


In [45]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense

# Define the function to create the neural network model
def create_nn_model(input_shape, learning_rate=0.001, num_layers=3, num_neurons=[32, 64, 32]):
    model = Sequential()
    model.add(Dense(num_neurons[0], input_shape=(input_shape,), activation='relu'))

    for i in range(1, num_layers):
        model.add(Dense(num_neurons[i], activation='relu'))

    model.add(Dense(5, activation='softmax'))

    optimizer = tf.keras.optimizers.Adam(learning_rate=learning_rate)
    model.compile(optimizer=optimizer, loss='sparse_categorical_crossentropy', metrics=['accuracy'])

    return model

# Assuming X_train_scaled and X_test_scaled are already defined
# Assuming y_train and y_test are already defined

# Define the parameters for creating the model
learning_rate = 0.001
num_layers = 3
num_neurons = [32, 64, 32]

# Create and compile the PSO-optimized model
pso_optimized_model = create_nn_model(X_train_scaled.shape[1], learning_rate, num_layers, num_neurons)
pso_optimized_model.fit(X_train_scaled, y_train, epochs=50, batch_size=32, verbose=0)
pso_loss, pso_accuracy = pso_optimized_model.evaluate(X_test_scaled, y_test)

print("PSO-Optimized Model Accuracy:", pso_accuracy)


10/10 [==============================] - 0s 2ms/step - loss: 0.4991 - accuracy: 0.8259
PSO-Optimized Model Accuracy: 0.8259385824203491
